前言：

关于RAG，我们使用Chroma储存向量。可以直接储存不使用客户端，但为了方便管理，建议初始化客户端。

什么是 Chroma 客户端（Client）？

    Chroma 客户端是一个用于与 Chroma 向量数据库进行交互的对象，它封装了与数据库相关的操作。例如，创建或打开数据库、创建集合、插入数据、执行搜索查询等。
    
        连接数据库：客户端用于连接和管理数据库的实例。
        
        管理集合（Collection）：Chroma 中的数据存储单元是集合（Collection），客户端可以帮助你创建、打开和管理这些集合。
        
        数据存储与查询：客户端提供方法来向集合中插入数据、执行向量检索查询等。
        
为什么需要初始化客户端？

    数据库连接管理：Chroma 数据库存储在文件系统中（如 SQLite、磁盘上的文件）。客户端负责打开和管理数据库连接，确保你能够有效地存取数据。客户端就像一个“管理器”，确保你与数据库的交互顺畅。
    
    集合管理：在 Chroma 中，数据被组织在 集合（Collection） 中。客户端提供了方法来创建、查询、更新和删除这些集合。你通过客户端操作集合而不需要直接与底层数据库交互。
    
    向量操作和检索：Chroma 主要用于存储和检索向量数据。客户端提供了插入（upsert）向量、执行近似搜索等功能，从而使得向量数据处理变得简单和高效。
    
    方便的 API：客户端封装了很多底层的操作，使得用户只需专注于更高层次的业务逻辑，而无需关心底层实现细节。
    
客户端与数据库文件

    如果不使用客户端，你的 Chroma 数据库将没有直接的接口进行操作，因为客户端就是你与数据库文件之间的桥梁。
    
    数据库存储位置：Chroma 会将数据存储在一个文件中，通常是 .db 文件（比如 SQLite 格式的数据库）。这个文件保存在你指定的路径下。如果没有客户端管理，你就无法访问和操作这个数据库文件。
    
    客户端的作用：客户端通过指定的路径打开或创建这个数据库文件，管理其中的集合、向量和其他数据。
    
以下内容参考Chroma官方网站教程。

（1）Chroma部署客户端：

In [ ]:
"""部署临时客户端"""
import chromadb
client = chromadb.Client()

In [ ]:
"""部署持久客户端"""
import chromadb
from chromadb.config import Settings
client = chromadb.PersistentClient(path="/path/to/save/to", settings=Settings(allow_reset=True))
# 用户和数据库默认即可
# settings=Settings(allow_reset=True)是为了方便重置客户端。

（2）Chroma的集合操作：

此处只需要简单了解，在构建RAG时使用的是LangChain官方网站中LangChain-RAG-Chroma的操作指南。

注：

Chroma数据库：

    Chroma 数据库是整个存储系统，用于保存和管理多个集合（collection）
    
    数据库可以持久化到本地或内存中，具体路径由 PersistentClient 或 Client 的 path 参数指定
    
Collection集合：

    集合是 Chroma 数据库中的一个逻辑分区，用于存储一组向量及其相关文档
    
    集合可以通过 vectorstore 对象来进行操作
    
vectorstore=Chroma(embedding_function=embeddings)

    是 Chroma 集合的接口，用于对集合执行操作，例如插入数据、删除数据或查询数据
    
    vectorstore 对应集合 accident_reports_collection。
    
    通过 vectorstore，你可以对 accident_reports_collection 执行插入、查询等操作

In [ ]:
"""Chroma的集合操作"""
# 集合的创建及获取
collection = client.create_collection(name="my_collection", embedding_function=emb_fn)
collection = client.get_collection(name="my_collection", embedding_function=emb_fn)
# 也可以创建or获取，以及删除
collection = client.get_or_create_collection(name="test") 
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
client.delete_collection(name="my_collection") 
# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

# 配置集合
collection = client.create_collection(
    name="my_collection", 
    embedding_function=emb_fn,
    metadata={
        "hnsw:space": "cosine",
        "hnsw:ef_search": 100
    }
)

# 添加数据
collection.add(
    documents=["lorem ipsum...", "doc2", "doc3", ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)
# 如果文档太大而无法使用所选的嵌入函数进行嵌入，则会引发异常
# 每个文档必须具有唯一关联的ID

# 更新数据
collection.update(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)
# 如果ID在集合中找不到，则会记录错误并忽略更新
# 更新现有项目or添加尚不存在的项目
collection.upsert(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)

# 删除项目
collection.delete(
    ids=["id1", "id2", "id3",...],
	where={"chapter": "20"}
)
# 使用ID删除项目

（3）查询并获取：（同样简单了解）

In [ ]:
collection.query(
    query_embeddings=[[11.1, 12.1, 13.1],[1.1, 2.3, 3.2], ...],
    n_results=10,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
)
# $contains":"search_string：查询包含search_string的元数据

（4）使用嵌入函数：（同上）

In [ ]:
"""Chroma中使用OpenAI的嵌入函数"""
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="YOUR_API_KEY",
                model_name="text-embedding-3-small"
            )